In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
print(tf.__version__)

In [ ]:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()

In [ ]:
train_image=train_image/255.0
test_image=test_image/255.0

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((train_image,train_label))
ds_test = tf.data.Dataset.from_tensor_slices((test_image,test_label))

In [ ]:
#定义分布式策略
strategy = tf.distribute.MirroredStrategy()
print("number of devices:{}".format(strategy.num_replicas_in_sync))

In [ ]:
#设置输入流水线
#当使用多个GPU训练模型师，可通过增加批次大小来有效利用额外的计算能力，通常，应使用适合GPU内存的批次大小，并相应调整学习率
BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA=64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA*strategy.num_replicas_in_sync

In [ ]:
train_dataset = ds_train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = ds_test.batch(BATCH_SIZE)

In [ ]:
# 生成模型
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32,3,activation='relu',input_shape=(28,28,1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])